<a href="https://colab.research.google.com/github/remanx2558/MIDBReviews/blob/master/RMDL_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install bert-for-tf2
!pip install sentencepiece

In [51]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [52]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os
import sys
import pickle
import itertools
import gensim
from sklearn.model_selection import train_test_split
from numpy import zeros
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

In [53]:
# Importing dataset
reviews_df = pd.read_csv(r"/content/drive/My Drive/malia/train.csv")
reviews_df.isnull().values.any()
reviews_df.sentiment= reviews_df.sentiment.fillna(0.0).astype(int)#this will conver float into int and also manage missing values
print(reviews_df.dtypes)

text         object
sentiment     int64
dtype: object


In [54]:
def clean(text):
    '''
    '''
    text = text.lower()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("should've", "should have")
    text = text.replace("should't", "should not")
    text = text.replace("should't've", "should not have")
    text = text.replace("would've", "would have")
    text = text.replace("would't", "would not")
    text = text.replace("would't've", "would not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace("'t", "")
    text = text.replace("'ve", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace(",", " , ")
    text = text.replace("´", "")
    text = text.replace("‘", "")
    text = text.replace("’", "")
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace("-", "")
    text = text.replace("–", "")
    text = text.replace("—", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.replace("\n", " \n ")
    text = text.replace("\n", "")
    text = text.replace("_", " _ ")
    text = text.replace("_", "")
    text = ''.join([i for i in text if not i.isdigit()])

    return text

texts = reviews_df['text'].values

cleaned_texts = [clean(r) for r in texts] 

reviews_df['text'] = cleaned_texts


In [55]:
# Shuffling data
reviews_df = reviews_df.sample(frac=1).reset_index(drop=True)

# Extracting all text
texts = reviews_df['text'].values
reviews_text = []

for p in zip(texts) : 
 reviews_text.append(p)



In [56]:
# Preprocessing training data


# Creating datasets
dict1 ={
    'reviews' : reviews_df['text'],
    'labels' : reviews_df['sentiment']
}
sentiment_df = pd.DataFrame.from_dict(dict1)


dict2 ={
    'reviews_text' : reviews_df['text']
}
reviews_text_df = pd.DataFrame.from_dict(dict2)

In [57]:

print(sentiment_df[:10])
print(reviews_text_df[:10])

                                             reviews  labels
0  a bunch of kids set up a theatre to have an al...       1
1  reading my review of the house that screamed ,...       1
2  the original lensman series of novels is a cla...       1
3  this movie is fun to watch .  if you liked dav...       0
4   .  .  . but itll make you wonder if we had an...       1
5  at least i was able to enjoy mocking the movie...       1
6  i love zombiemovies and i love amateurproducti...       1
7  please make me forget .  please .  please .  t...       1
8  alright this was quite a sensitive little numb...       0
9  the gang of roses .  every rose has its thorns...       0
                                        reviews_text
0  a bunch of kids set up a theatre to have an al...
1  reading my review of the house that screamed ,...
2  the original lensman series of novels is a cla...
3  this movie is fun to watch .  if you liked dav...
4   .  .  . but itll make you wonder if we had an...
5  at least

In [58]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [59]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [60]:
reviews = []
sentences = list(sentiment_df['reviews'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [61]:
print(sentiment_df.columns.values)

['reviews' 'labels']


In [62]:
sentiment_df.labels.unique()

array([1, 0])

In [63]:
print(reviews[20])

this piece of filth is virtually impossible to follow the sound is crap the picture quality goes from bad to worse to good to bad again things happen for no apparent reason characters appear and disappear was the director suffering from massive brain injury during its production poor film making aside the story is vile just sick evil sht if you like rape murder and self harm this is right up your alley and if simulated scenes of murder are not enough you can enjoy clips of actual people being executed watched almost all of it but had to turn off after seen someones brains blown out never before have seen film that left me feeling so ashamed and dirty 


In [64]:
y = sentiment_df['labels']


In [65]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews[10]))

In [67]:
def tokenize_reviews(reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews))

In [68]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [69]:
print(tokenized_reviews[10])

[2096, 2087, 1997, 1996, 3185, 2003, 2200, 5515, 4509, 1996, 12849, 19603, 14574, 3496, 2003, 2209, 2039, 2021, 2025, 4895, 16344, 5657, 12849, 19603, 2375, 2758, 2008, 2019, 4111, 2442, 2022, 9715, 2043, 1996, 6085, 12817, 2009, 3096, 1996, 12849, 19603, 14574, 3496, 2003, 8321, 2004, 3087, 4282, 2040, 2038, 2464, 2028, 2030, 2038, 2464, 1996, 9004, 2050, 2143, 4760, 12849, 19603, 14574, 1999, 2029, 1996, 4176, 3759, 2003, 3013, 1998, 1996, 9686, 7361, 3270, 12349, 3013, 2041, 2096, 2009, 2003, 2145, 4142, 9715, 1998, 5525, 6114, 2057, 2442, 3342, 2008, 2381, 2003, 2517, 2011, 1996, 5125, 2015, 2003, 2028, 2130, 3039, 2000, 2130, 2228, 2008, 2672, 1996, 13157, 2020, 2157, 2515, 2025, 2009, 2360, 2505, 2008, 1996, 13157, 2018, 29131, 2023, 13925, 3412, 14574, 1998, 1996, 5181, 2024, 2145, 12560, 2009, 2130, 2651]


In [76]:
reviews_with_len = [[review, y[i]]
                 for i, review in enumerate(tokenized_reviews)]

In [77]:
import random
random.shuffle(reviews_with_len)

In [78]:
print(reviews_with_len[:2])

[[[3731, 3423, 2038, 2357, 2049, 5725, 1998, 2003, 3753, 2005, 1996, 8659, 2341, 1998, 16215, 10369, 22889, 7361, 2009, 2038, 2580, 2043, 2023, 2265, 2034, 4836, 2471, 2176, 2161, 2067, 2009, 2001, 14742, 14308, 2012, 1996, 3423, 2291, 2029, 2035, 5889, 4025, 2000, 2202, 6620, 1999, 17274, 2009, 2001, 6057, 24908, 1998, 2000, 2070, 6698, 25854, 1996, 3494, 6791, 2020, 11701, 1998, 2000, 2019, 6698, 2613, 1999, 2037, 13954, 2015, 1996, 4424, 7615, 1998, 4023, 2020, 3132, 1998, 14742, 7628, 16208, 2003, 1998, 2001, 3376, 2004, 1999, 2060, 2186, 2016, 4194, 2021, 2003, 2085, 7944, 2000, 1996, 2896, 11143, 1997, 2865, 4730, 1997, 3348, 1998, 4808, 7628, 2003, 2019, 6581, 3883, 1998, 3791, 2062, 6540, 4132, 2084, 2023, 2537, 10731, 4748, 9103, 4103, 12083, 10054, 2003, 2019, 6581, 3364, 2040, 2038, 2013, 1996, 2420, 1997, 11999, 2000, 2023, 2537, 2218, 2010, 2219, 1999, 1996, 2492, 1997, 4024, 2467, 4760, 1996, 8562, 1998, 26438, 3772, 1997, 1996, 2537, 2952, 11332, 2003, 6057, 1998, 14742,

In [79]:
reviews_with_len.sort(key=lambda x: x[2])

IndexError: ignored

In [80]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [81]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [82]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [83]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 1614), dtype=int32, numpy=
 array([[ 3731,  3423,  2038, ...,     0,     0,     0],
        [ 3100,  2023,  2038, ...,     0,     0,     0],
        [ 1045,  2034,  2387, ...,     0,     0,     0],
        ...,
        [ 3958, 12385,  3240, ...,     0,     0,     0],
        [10047,  2183,  2000, ...,     0,     0,     0],
        [10882,  3527,  2003, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 0], dtype=int32)>)

In [84]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [85]:
import tensorflow as tf
import keras

In [86]:

class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        #self.rnn_layer = layers.Bidirectional(self.rnn_layer.layers.LSTM(32)),
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        concatenated = tf.concat([l_1, l_2, l_3,], axis=-1) # (batch_size, 4 * rmdl_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [87]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [88]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [89]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [90]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
704/704 [==============================] - 140s 199ms/step - loss: 0.3605 - accuracy: 0.8303
Epoch 2/5
704/704 [==============================] - 90s 128ms/step - loss: 0.1321 - accuracy: 0.9529
Epoch 3/5
704/704 [==============================] - 91s 129ms/step - loss: 0.0463 - accuracy: 0.9829
Epoch 4/5
704/704 [==============================] - 88s 126ms/step - loss: 0.0292 - accuracy: 0.9894
Epoch 5/5
704/704 [==============================] - 89s 126ms/step - loss: 0.0204 - accuracy: 0.9925


In [91]:
loss, accuracy = text_model.evaluate(test_data)
print('Test Loss: {}'.format(loss))
print('Test Accuracy: {}'.format(accuracy))

78/78 [==============================] - 4s 46ms/step - loss: 1.0204 - accuracy: 0.8241
Test Loss: 1.0204379558563232
Test Accuracy: 0.8241186141967773
